In [ ]:
# Import necessary libraries
!pip install patool==1.12
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from google.colab import drive
import patoolib

# Mount Google Drive
drive.mount('/content/drive')

# Define the directory containing the dataset
data_dir = '/content/extracted_images/leapGestRecog'


# # 1. Rotation Only
# data_augmentation = tf.keras.preprocessing.image.ImageDataGenerator(
#     validation_split=0.3,  # Reserve 30% of the dataset for validation
#     rotation_range=20,     # Rotate images by up to 20 degrees
#     rescale=1./255         # Scale images to [0, 1] range
# )

# 2. Flipping Only
data_augmentation = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.3,  # Reserve 30% of the dataset for validation
    horizontal_flip=True,  # Randomly flip images horizontally
    rescale=1./255         # Scale images to [0, 1] range
 )

# 3. Scaling Only
# data_augmentation = tf.keras.preprocessing.image.ImageDataGenerator(
#     validation_split=0.3,  # Reserve 30% of the dataset for validation
#     zoom_range=0.2,        # Zoom in/out by up to 20%
#     rescale=1./255         # Scale images to [0, 1] range
# )


train_dataset = data_augmentation.flow_from_directory(
    directory=data_dir,
    target_size=(224, 224),  # Resize images to 224x224 pixels
    batch_size=32,           # Load images in batches of 32
    class_mode='categorical',
    subset='training',       # Use the 'training' subset
    seed=123
)


validation_dataset = data_augmentation.flow_from_directory(
    directory=data_dir,
    target_size=(224, 224),  # Resize images to 224x224 pixels
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    seed=123
)

# class names to verify labels
class_names = list(train_dataset.class_indices.keys())
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")

# Load the pre-trained VGG16 model with custom layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the layers in the base model


model = models.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),  # Explicit input shape layer
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the VGG16 model with the selected data augmentation
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10  # Adjust the number of epochs based on your requirements
)

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_dataset, verbose=2)
print(f"Validation accuracy with the applied augmentation: {val_acc:.4f}")

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy with Data Augmentation')
plt.show()
